In [1]:
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# elem.send_keys(Keys.ENTER)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import re
import time
from datetime import datetime
import pickle

import pymysql 
import pandas as pd 
import numpy as np
from tqdm import tqdm

# # save data
# with open('urls.pickle','wb') as fw:
#     pickle.dump(myset.urls, fw)

# # load data
# with open('urls.pickle', 'rb') as fr:
#     urls = pickle.load(fr)

In [2]:
class createDB():
    def __init__(self, genres={'소설':'01', '시에세이':'03', '인문':'05', '경제경영':'13', 
                                '자기계발':'15', '정치사회':'17', '역사문화':'19', 
                                '예술대중문화':'23', '과학':'29' }):
        self.genres = genres
        self.urls, self.dfs = {}, {}
        
    def open_driver(self, url, scroll=False):
        user_agent = 'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
        options = webdriver.ChromeOptions()
        options.add_argument(user_agent)
        options.add_argument('headless') # 페이지 안 열기 
        options.add_argument('window-size=1920x1080') # 사이즈 지정
        options.add_argument("disable-gpu") # 안 보이게
        self.driver = webdriver.Chrome(options=options)
        self.driver.implicitly_wait(1)
        self.driver.get(url)
        time.sleep(0.5)
        
        if scroll:
            prev_height = self.driver.execute_script('return document.body.scrollHeight')
            while True:
                self.driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                time.sleep(1)
                curr_height = self.driver.execute_script('return document.body.scrollHeight')
                if prev_height == curr_height: break
                prev_height = curr_height
        
    def save_urls(self, genre):
        self.urls[genre] = {}
        tqdm_pages = tqdm(range(20,0,-1))
        for page in tqdm_pages: # 한 페이지 당 50권, 20페이지 반복(1000위까지)
            url = 'https://product.kyobobook.co.kr/category/KOR/{}#?page={}&type=best&per=50'.format(self.genres[genre], page)
            self.open_driver(url)
            best = self.driver.find_element_by_css_selector('#homeTabBest')
            elems = best.find_elements_by_class_name('prod_info_box')
            for i, elem in enumerate(elems):
                print('page-i:', page, i)
                try:
                    title = elem.find_element_by_css_selector('.prod_name').text
                    rank = elem.find_element_by_class_name('text').text
                    author = elem.find_element_by_css_selector('.prod_author > a').text
                    temp_url = elem.find_element_by_class_name('prod_info')
                    url = temp_url.get_attribute('href')
                    temp_rate = elem.find_element_by_class_name('review_klover_box')
                    rate = temp_rate.find_element(By.TAG_NAME, 'span').text
                    temp_count = elem.find_element_by_css_selector('.review_desc').text
                    count = re.sub(r'[^0-9]', '', temp_count)
                    
                    self.urls[genre][title] = {}
                    self.urls[genre][title]['url'] = url
                    self.urls[genre][title]['rank'] = int(rank)
                    self.urls[genre][title]['author'] = author
                    self.urls[genre][title]['rate'] = float(rate)
                    self.urls[genre][title]['count'] = int(count)
                except: print('pass')
                    
            tqdm_pages.set_description(genre+' '+str(page)+' urls')
        tqdm_pages.close()

    def connect_mysql(self, connect=True):
        if connect:
            self.db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234', db='pagepalette', 
                                      charset='utf8', cursorclass=pymysql.cursors.DictCursor)
            self.cursor = self.db.cursor()
        else:
            self.db.commit()
            self.db.close
            
    def create_tbl(self, genre):
        self.connect_mysql()
        sql = """CREATE TABLE IF NOT EXISTS gyobo_{}(
                title VARCHAR(100), author VARCHAR(100),
                genre_1 VARCHAR(200), genre_2 VARCHAR(300), 
                brank SMALLINT, rate DOUBLE, count SMALLINT, isbn BIGINT
                );""".format(genre)
        self.cursor.execute(sql)
        self.connect_mysql(False)
    
    def fill_tbl(self, genre):
        self.connect_mysql()
        tqdm_book = tqdm(self.urls[genre].items())
        for book_title, book_info in tqdm_book:
            try: self.open_driver(book_info['url'], scroll=True)
            except: continue
                
            title = book_title
#             title = self.driver.find_element_by_class_name('prod_title').text
            author, rank = book_info['author'], book_info['rank']
            rate = book_info['rate']
            count = book_info['count'] if book_info['count'] is not None else 'Null'
            
            elems = self.driver.find_elements_by_class_name('category_list_item')
            genre_1, genre_2 = '', ''
            for elem in elems:
                gs = elem.find_elements_by_class_name('intro_category_link')
                for i, g in enumerate(gs):
                    if i==1: genre_1 += g.text+', '
                    elif i==2 or i==3: genre_2 += g.text+', '
            if len(genre_1)==0: continue
                        
            isbn_box = self.driver.find_element_by_class_name('product_detail_area.basic_info')
            isbn = int(isbn_box.find_element(By.TAG_NAME, 'td').text)
            
            sql = '''INSERT INTO gyobo_{0} VALUES(
            "{1}", "{2}", "{3}", "{4}", {5}, {6}, {7}, {8});
            '''.format(genre, title, author, genre_1, genre_2, rank, rate, count, isbn)
            self.cursor.execute(sql)
            self.db.commit()
            tqdm_book.set_description(genre+' insert data')
        self.connect_mysql(False)
        tqdm_book.close()
        
    def get_df(self, genre):
        self.connect_mysql(True)
        sql = 'SELECT * FROM gyobo_{}'.format(genre)
        self.cursor.execute(sql)
        self.dfs[genre] = pd.DataFrame(self.cursor.fetchall())
        self.dfs[genre].to_csv('gyobo_{}.csv'.format(genre), index=False)
        self.connect_mysql(False)
        print('save gyobo_'+genre+'.csv')
    
    def auto(self, genre, opt=True):
        if opt: self.save_urls(genre)
        self.create_tbl(genre)
        self.fill_tbl(genre)
        self.get_df(genre)

In [3]:
myset = createDB()
print(myset.genres)
# for genre in myset.genres.keys():
#     myset.auto(genre)

{'소설': '01', '시에세이': '03', '인문': '05', '경제경영': '13', '자기계발': '15', '정치사회': '17', '역사문화': '19', '예술대중문화': '23', '과학': '29'}


In [4]:
with open('urls_gyobo.pickle', 'rb') as fr:
    urls = pickle.load(fr)

for genre in urls.keys():
    print(genre, len(urls[genre].keys()))

소설 863
시에세이 945
인문 890
경제경영 990
자기계발 943
정치사회 983
역사문화 998
예술대중문화 990
과학 978


In [ ]:
myset.urls = urls
for genre in urls.keys():
    myset.auto(genre, opt=False)

소설 insert data: 100%|████████████████████████████████████████████████████████████| 863/863 [1:55:47<00:00,  8.05s/it]s]


save gyobo_소설.csv


시에세이 insert data: 100%|████████████████████████████████████████████████████████| 945/945 [2:08:18<00:00,  8.15s/it]t/s]


save gyobo_시에세이.csv


인문 insert data: 100%|████████████████████████████████████████████████████████████| 890/890 [2:03:03<00:00,  8.30s/it]s]


...

자기계발 insert data: 100%|████████████████████████████████████████████████████████| 943/943 [2:03:38<00:00,  7.87s/it]t/s]


save gyobo_자기계발.csv


정치사회 insert data: 100%|████████████████████████████████████████████████████████| 983/983 [2:04:38<00:00,  7.61s/it]t/s]


save gyobo_정치사회.csv


역사문화 insert data:  42%|████████████████████████▎                                 | 419/998 [52:38<59:22,  6.15s/it]t/s]

In [5]:
myset.urls = urls
for genre in ['역사문화','예술대중문화','과학']:
    myset.auto(genre, opt=False)

역사문화 insert data: 100%|████████████████████████████████████████████████████████| 998/998 [2:08:33<00:00,  7.73s/it]t/s]


save gyobo_역사문화.csv


예술대중문화 insert data: 100%|████████████████████████████████████████████████████| 990/990 [2:06:24<00:00,  7.66s/it]?it/s]


save gyobo_예술대중문화.csv


과학 insert data: 100%|████████████████████████████████████████████████████████████| 978/978 [2:09:22<00:00,  7.94s/it]s]


save gyobo_과학.csv


In [7]:
with open('urls_gyobo_etc_travel.pickle', 'rb') as fr:
    urls = pickle.load(fr)

for genre in urls.keys():
    print(genre, len(urls[genre].keys()))

여행 100
여행에세이 74


In [8]:
urls

{'여행': {'나를 채우는 여행의 기술': {'url': 'https://product.kyobobook.co.kr/detail/S000201094491',
   'rank': '51',
   'author': '알랭 드 보통',
   'rate': '10',
   'count': '8'},
  '리얼 다낭: 호이안 후에(2023~2024)': {'url': 'https://product.kyobobook.co.kr/detail/S000200219182',
   'rank': '52',
   'author': '배나영',
   'rate': '9.17',
   'count': '11'},
  '프렌즈 스페인 포르투갈(2022~2023)': {'url': 'https://product.kyobobook.co.kr/detail/S000061694075',
   'rank': '53',
   'author': '박현숙',
   'rate': '9.5',
   'count': '17'},
  '프렌즈 캐나다(2022~2023)': {'url': 'https://product.kyobobook.co.kr/detail/S000061350537',
   'rank': '54',
   'author': '이주은 외',
   'rate': '10',
   'count': '23'},
  '특별하게 싱가포르(2023~2024)': {'url': 'https://product.kyobobook.co.kr/detail/S000201351847',
   'rank': '55',
   'author': '설혜원',
   'rate': '10',
   'count': '3'},
  '치앙마이 홀리데이(2023~2024)': {'url': 'https://product.kyobobook.co.kr/detail/S000200177666',
   'rank': '56',
   'author': '박애진',
   'rate': '9.71',
   'count': '12'},
  '리얼 교토: